# Deep Reinforcement Learning for Snake

The `SnakeEnv` is defined in `env.py`. The Neural Network we use to approximate $Q(a,s)$ is defined in `DQN.py`.

In [12]:
# importing cell
from env import *
from DQN import *
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt

We take ideas from the <a href="https://arxiv.org/pdf/1312.5602.pdf">Playing Atari with Deep Reinforcement Learning</a> paper. The idea is, always in RL, to approximate and learn the optimal action-value function $Q^*(s,a)$ which is defined as

\begin{equation*}
Q^*(s,a) := \max_{\pi} \mathbb{E} \left [ R_t | s_t = s,a_t=a,\pi \right ]
\tag{1}
\end{equation*}

That is, the optimal value for an action given a certain state is the expected return of taking that action in that state, assuming an ideal policy that maximizes such expected return.
The return is defined as the sum of all rewards from time $t$ up to the end of the episode, discounted with a factor $\gamma$ that regulates the importance of rewards far away in the future: $R_t = \sum_{t'=t}^T \gamma^{t' - t}r_{t'}$. With this definition, and by defining the environment as $\mathcal{E}$, we can rewrite the definition of $Q^*$, using the fundamental recurrence relation that is known as _Bellman's optimality equation_:

\begin{equation*}
Q^*(s,a) = \mathbb{E}_{s' \sim \mathcal{E}} \left [ r + \gamma \max_{a'} Q^*(s',a') | s,a \right ]
\tag{2}
\end{equation*}

In cases where this is computationally feasible, one would often implement the _value iteration algorithm_ by using the equation (2) as a update rule to find the policy that maximizes the value for each state. But that would require keeping a table of values for each $(s,a)$ pair; that is, in many cases, intractable, so an alternative is to use an _approximation_ of the function $Q^*(s,a)$ that computes the value given a certain state using a number of paramters which hopefully is much smaller than the cardinality of the set of all possible states and actions.

## Q-network

In our case, the approximation function is a Convolutional Neural Network (CNN) that takes the screen of the Snake game (represented by a $W \times H$ matrix of integers, where -1 is the food, 0 represent empty spaces and integers 1,2,3,... represent pieces of the snake, 1 being the head) and returns 4 values, one for each of the possible actions.

We refer to the neural network function approximator as *Q-network* with weights $\theta$.

In [13]:
model = DQN()
env = SnakeEnv()

obs,_ = env.reset()

print('Screen matrix: ')
print(obs['screen'])
print('\n\nDirection vector: ')
print(obs['direction'])
print('\n\nQ-network output: ')
print(model(obs))

Screen matrix: 
[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 1  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0 -1  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]]


Direction vector: 
[0. 1. 0. 0.]


Q-network output: 
tensor([[ 0.0200, -0.1350,  0.0019,  0.0600]], grad_fn=<AddmmBackward0>)


The model is a mixture of a few convolutional layers applied to the screen input and a few feed forward layers that take the flattened output of the convolutional layers and the direction vector:

<img src='./img/dqn_torchviz.png' class='center' height=800>


## Training

The Q-network can be trained to minimize the following loss:

\begin{align*}

L_i (\theta_i) = & \mathbb{E}_{s,a \sim \rho (\cdot)} \left [ \left ( y_i - Q(a,s;\theta_i) \right )^2 \right ] \\

y_i := & \mathbb{E}_{s' \sim \mathcal{E}} \left [ r + \gamma \max_{a'} Q(s',a';\theta_{i-1}) | s,a \right ]

\tag{3}
\end{align*}

where $\rho$ is a _behaviour policy_, i.e. a distribution over actions (tipically $\epsilon$-greedy), and $y_i$ is the _TD target_, which as we can see, depends on the previous estimation of the $Q$ function. This is a key aspect in all _Temporal Difference_ algorithm, in which the estimation of $R_t$ comes from a previous estimation, a technique known as _bootstrapping_.

When we update the loss function, trying to minimize it via gradient descent, we compute the following gradient w.r.t. the weights:

\begin{equation*}

\nabla _{\theta_i} L_i (\theta_i) = \mathbb{E}_{s,a \sim \rho(\cdot);s' \sim \mathcal{E}} \left [ \left( \underbrace{r + \gamma \max_{a'} Q(s',a';\theta_{i-1})}_{TD \; target} - Q(s,a;\theta_i) \right ) \nabla_{\theta_i} Q(s,a;\theta_i) \right ]

\tag{4}
\end{equation*}


The loss function is defined as the expectation over all states, but it's often convenient to perform a _stochastic gradient descent_, that is minimize not the loss function averaged through the whole dataset but only on a smaller batch size, whose expectation value is, in average, the same as the "total" one.

If, furthermore, we set the batch size to be 1 (that is, to update after every $(s,a,r,s')$ occurrence sampled from the environment and the behaviour distribution) and we update the weights at every iteration, we arrive at the famous _Q-learning_ algorithm.

The Q-learning algorithm is **off policy**: it learns about the greedy policy $a = \max_a Q(s,a;\theta)$, while following a policy that is generally not completely greedy, but often $\epsilon$ greedy.